In [1]:
import pandas as pd

df = pd.read_csv('data/combined_data_sorted.csv')
# This converts the actual page number to its label importance
df['page'] = 50 - df['page']

from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2, random_state = 0)
df_test, df_valid = train_test_split(df_test, test_size = 0.5, random_state = 0)

print(len(df_train.index))
print(len(df_test.index))
print(len(df_valid.index))

df_train = df_train.sort_values(by = ['qid', 'page'], ascending = [True, False])
df_test = df_test.sort_values(by = ['qid', 'page'], ascending = [True, False])
df_valid = df_valid.sort_values(by = ['qid', 'page'], ascending = [True, False])

# df_train.to_csv('data/train.csv', index = False)
# df_test.to_csv('data/test.csv', index = False)
# df_valid.to_csv('data/valid.csv', index = False)

157989
19749
19749


In [4]:
import xgboost as xgb

dtrain = xgb.DMatrix('data/train.txt')
dtrain_group = pd.read_csv('data/train.csv').groupby(['qid']).size().values
dtrain.set_group(dtrain_group)

dtest  = xgb.DMatrix('data/test.txt')
dtest_group = pd.read_csv('data/test.csv').groupby(['qid']).size().values
dtest.set_group(dtest_group)

dvalid = xgb.DMatrix('data/valid.txt')
dvalid_group = pd.read_csv('data/valid.csv').groupby(['qid']).size().values
dvalid.set_group(dvalid_group)

params = {
    'objective': 'rank:pairwise',
    'eval_metric': 'ndcg',
    'max_depth': 10
}

bst = xgb.train(params, dtrain, num_boost_round = 10, evals = [(dtrain, 'train'), (dvalid, 'eval')])
print(bst.predict(dtest))

[16:21:23] 157989x15 matrix with 2211846 entries loaded from data/train.txt
[16:21:24] 19749x15 matrix with 276486 entries loaded from data/test.txt
[16:21:24] 19749x15 matrix with 276486 entries loaded from data/valid.txt
[0]	train-ndcg:0.985411	eval-ndcg:1.00324
[1]	train-ndcg:0.984895	eval-ndcg:0.997044
[2]	train-ndcg:0.984488	eval-ndcg:0.996048
[3]	train-ndcg:0.984629	eval-ndcg:0.994509
[4]	train-ndcg:0.985558	eval-ndcg:0.995685
[5]	train-ndcg:0.985012	eval-ndcg:0.996697
[6]	train-ndcg:0.985374	eval-ndcg:0.992829
[7]	train-ndcg:0.986042	eval-ndcg:0.992206
[8]	train-ndcg:0.986485	eval-ndcg:0.99134
[9]	train-ndcg:0.985809	eval-ndcg:0.992673
[ 0.39328828  0.2774906   0.50972015 ... -0.14677787 -0.16700876
 -0.16700876]


In [3]:
from xgboost import plot_tree
import matplotlib.pyplot as plt

columns = pd.read_csv('data/train.csv').columns[1:-1]
feature_importance = {
    columns[0]: bst.get_fscore()['f1'],
    columns[1]: bst.get_fscore()['f2'],
    columns[2]: bst.get_fscore()['f3'],
    columns[3]: bst.get_fscore()['f4'],
    columns[4]: bst.get_fscore()['f5'],
    columns[5]: bst.get_fscore()['f6'],
    columns[6]: bst.get_fscore()['f7'],
    columns[7]: 0,
    columns[8]: bst.get_fscore()['f9'],
    columns[9]: bst.get_fscore()['f10'],
    columns[10]: 0,
    columns[11]: bst.get_fscore()['f12'],
    columns[12]: bst.get_fscore()['f13'],
    columns[13]: bst.get_fscore()['f14']
}
print("Feature Importance:")
feature_importance

Feature Importance:


{'titleScore': 418,
 'price': 2420,
 'salesCount': 1085,
 'goodComment': 1239,
 'badComment': 170,
 'picComment': 617,
 'categoryId': 236,
 'downTime': 0,
 'freeDelivery': 214,
 'isNewProduct': 30,
 'commonweal': 0,
 'taojinbi': 143,
 'shopCreditScore': 1193,
 'shopServiceScore': 388}